In [1]:
import gurobipy as gp
from gurobipy import GRB

## Uncapacitated and Capacitated Lot-Sizing Problem (LS-U and LS-C)

In [2]:
# Initialize model
m = gp.Model("ls")

capacitated = False

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-30


### Define Variables

In [3]:
# Time (e.g. months)
n = 8
t = range(0, n)

# Costs for production of the product for each month
p = [100 for i in t]

# Costs for storing a product for each month
h = [5 for i in t]

# Costs for armoring the machines for each month
q = [5000 for i in t]

# Demand of products for each month
d = [400, 400, 800, 800, 1200, 1200, 1200, 1200]

# Producttion Amount for each month
x = m.addVars(n, name="x", vtype=GRB.INTEGER)

# Storage Amount for each month
s = m.addVars(n + 1, name="s", vtype=GRB.INTEGER)

# Production Preparation neccessary for each month (0 or 1)
y = m.addVars(n, name="y", vtype=GRB.BINARY)

# Production Capacity for each month (in this case unbounded -> LS-U)
if capacitated:
    M = [7000, 0, 0, 0, 0, 0, 0, 0]
else:
    M = m.addVars(n, name="M", vtype=GRB.INTEGER)


### Define Objective Function

In [4]:
# Objective Function to minimize the overall costs
m.setObjective(gp.quicksum(p[i] * x[i] + q[i] * y[i] + h[i] * s[i+1] for i in t), GRB.MINIMIZE)

### Define Constraints

In [5]:
# Stored products from the previous month plus the amount of products produced in the current 
# month must fulfill the demand while the rest of the products must be stored for the next month
m.addConstrs((s[i-1] + x[i-1] == d[i-1] + s[i] for i in range(1, n+1)), "c1")
m.addConstr((s[0] + x[0] == d[0] + s[1]), "c2")

# The amount of products stored in the first month must be equal to 200 (Initial stock)
m.addConstr(s[0] == 200, "c3")
# The amount of products stored in the last month must be equal to 0 (Final stock)
m.addConstr(s[n] == 0, "c4")

# If products are being produced in the current month the machines must be prepared
m.addConstrs((x[i] <= M[i]*y[i] for i in t), "c5")

# There cant be negative amount of products stored in the warehouse or produced
m.addConstrs((x[i] >= 0 for i in t), "c6")
m.addConstrs((s[i] >= 0 for i in t), "c7")
m.addConstrs((M[i] >= 0 for i in t), "c8");

### Start Model Optimization

In [6]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 35 rows, 33 columns and 53 nonzeros
Model fingerprint: 0xa7ef1cf7
Model has 8 quadratic constraints
Variable types: 0 continuous, 33 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [5e+00, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 29 rows and 4 columns
Presolve time: 0.01s
Presolved: 30 rows, 53 columns, 74 nonzeros
Presolved model has 16 SOS constraint(s)
Variable types: 0 continuous, 53 integer (16 binary)
Found heuristic solution: objective 859000.00000
Found heuristic solution: objective 830000.00000
Found heuristic solution: objective 822000.00000

Root relaxation: objective 7.170501e+05, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Cu

### Evaluation

In [7]:
for v in m.getVars():
    if v.varName.startswith("x"):
        print("Produce {} products in month {}".format(v.x, v.varName[1:]))

print("--------------------------------")
print("Total cost: {}".format(m.objVal))

Produce 600.0 products in month [0]
Produce 0.0 products in month [1]
Produce 1600.0 products in month [2]
Produce 0.0 products in month [3]
Produce 1200.0 products in month [4]
Produce 1200.0 products in month [5]
Produce 1200.0 products in month [6]
Produce 1200.0 products in month [7]
--------------------------------
Total cost: 736000.0
